In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/PHD/SemEval2023/SemEval2022-Task10/private_space

/content/gdrive/MyDrive/PHD/SemEval2023/SemEval2022-Task10/private_space


In [3]:
# use github on google colab: https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

In [4]:
!git config --global user.name "Pakawat Nakwijit"
!git config --global user.email "p.nakwijit@qmul.ac.uk"

In [12]:
!git add ../Results

In [13]:
!git 

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   ../Results/1b_lexbert_dev.csv
	new file:   ../Results/1b_lexbert_test.csv
	new file:   ../Results/1c_lexbert_dev.csv
	new file:   ../Results/1c_lexbert_test.csv



In [14]:
!git commit -m "task b, c predictions"

[main 07184f3] task b, c predictions
 4 files changed, 2916 insertions(+)
 create mode 100644 Results/1b_lexbert_dev.csv
 create mode 100644 Results/1b_lexbert_test.csv
 create mode 100644 Results/1c_lexbert_dev.csv
 create mode 100644 Results/1c_lexbert_test.csv


'Augmentation with GPT-J.ipynb'      'Incorporate BERT TaskB.ipynb'
'Augmentation with TweetBERT.ipynb'  'Incorporate BERT TaskC.ipynb'
'Augmentation w  PMI.ipynb'          'Lexicon Construciton [PMI].ipynb'
'Augmentation w  XAI.ipynb'          'Lexicon Construciton [Shapley].ipynb'
 Data/                               'Logistic Regression with BoW.ipynb'
'Data Exploration.ipynb'              Models/
'Evaluation TaskB.ipynb'              Results/
'Evaluation TaskC.ipynb'             'Run BERT.ipynb'
'Fine-tuning BERT TaskA.ipynb'        Temo.ipynb
'Fine-tuning BERT TaskB C.ipynb'


In [ ]:
# !git push origin main

In [ ]:
import json

with open('../Results/TaskB/lexicon_pmi.json') as fin:
    lexicon_pmi = json.load(fin)

with open('../Results/TaskB/lexicon_shapley.json') as fin:
    lexicon_shapley = json.load(fin)

# with open('../Results/TaskB/lexicon_shapley_augmented_v1.json') as fin:
#     lexicon_shapley_augmented_v1 = json.load(fin)

# with open('../Results/TaskB/lexicon_shapley_augmented_v2_sqrt.json') as fin:
#     lexicon_shapley_augmented_v2_sqrt = json.load(fin)

with open('../Results/TaskB/lexicon_shapley_augmented_v2_shap.json') as fin:
    lexicon_shapley_augmented_v2_shap = json.load(fin)

with open('../Results/TaskB/lexicon_shapley_augmented_v3_gptj.json') as fin:
    lexicon_shapley_augmented_v3_gptj = json.load(fin)

with open('../Results/TaskB/lexicon_shapley_augmented_v4_tweetbert.json') as fin:
    lexicon_shapley_augmented_v4_tweetbert = json.load(fin)

In [ ]:
# (lexicon_shapley_augmented_v1['1. threats, plans to harm and incitement']["sister"])

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [ ]:
import pandas as pd

target_column = "label_category"
data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")
data = data[data[target_column]!="none"]

label_values = [
    '1. threats, plans to harm and incitement',
    '2. derogation',
    '3. animosity',
    '4. prejudiced discussions',
]

In [ ]:
label_map = {
    0: '1. threats, plans to harm and incitement',
    1: '2. derogation', 
    2: '3. animosity', 
    3: '4. prejudiced discussions',
    '1. threats, plans to harm and incitement':0,
    '2. derogation':1,
    '3. animosity':2,
    '4. prejudiced discussions':3,
}

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the stopwords
stoplist = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import string

def preprocess(sent, **kwargs):
  sent = sent.lower()
  words = nltk.word_tokenize(sent)

  # Remove Stop words
  words = [w for w in words if w not in stoplist]

  # Remove Punctuation
  words = [w for w in words if w not in string.punctuation]
  
  # Lematization
  if "lemma" in kwargs and kwargs["lemma"]:
    words = [lemmatizer.lemmatize(w) for w in words]
    
  return words

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

## Logistic Regression with all words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def logistic_preprocess(sent):
  words = preprocess(sent, lemma=True)
  return words

def dummy_word_tokenize(sent):
  return sent

vectorizer = CountVectorizer(
    preprocessor=logistic_preprocess,
    tokenizer=dummy_word_tokenize
)

In [ ]:
X_train = vectorizer.fit_transform(train["text"].values)
X_test = vectorizer.transform(test["text"].values)

In [ ]:
y_train = [label_map[l] for l in train[target_column].values]
y_test = [label_map[l] for l in test[target_column].values]

In [ ]:
"#Vocab", len(vectorizer.get_feature_names_out())

('#Vocab', 7574)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

model = LogisticRegression(random_state=42, penalty="l2")
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
print("F1: %.3f" % metrics.f1_score(y_test, y_predict, average='macro'))
print("Acc: %.3f" % metrics.accuracy_score(y_test, y_predict))

p, r, f1, sup = metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")
print("P: %.3f" % p)
print("R: %.3f" % r)

F1: 0.445
Acc: 0.504
P: 0.526
R: 0.415


In [ ]:
from tqdm import tqdm
import numpy as np


def cross_validation(data, fold=5):
  N = int(len(data)/fold)

  _metrics = {
      "acc":[],
      "p":[],
      "r":[],
      "f1":[]
  }

  for k in tqdm(range(fold), total=fold):
    test = data.iloc[k*N:(k+1)*N]
    train = pd.concat([data.iloc[:k*N], data.iloc[(k+1)*N:]])
    
    vectorizer = CountVectorizer(
        preprocessor=logistic_preprocess,
        tokenizer=dummy_word_tokenize
    )

    X_train = vectorizer.fit_transform(train["text"].values)
    X_test = vectorizer.transform(test["text"].values)

    y_train = [label_map[l] for l in train[target_column].values]
    y_test = [label_map[l] for l in test[target_column].values]

    model = LogisticRegression(random_state=42, penalty="l2")
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)

    # f1 = metrics.f1_score(y_test, y_predict, average='macro')
    acc = metrics.accuracy_score(y_test, y_predict)
    p, r, f1, sup = metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")

    _metrics["acc"].append(acc)
    _metrics["f1"].append(f1)
    _metrics["p"].append(p)
    _metrics["r"].append(r)
  
  print("")
  for m in _metrics:
    print(f"{m}: {np.mean(_metrics[m]):.3f} ± {np.std(_metrics[m]):.3f}")
  return ;

cross_validation(data)

100%|██████████| 5/5 [00:15<00:00,  3.09s/it]


acc: 0.497 ± 0.023
p: 0.487 ± 0.037
r: 0.391 ± 0.033
f1: 0.415 ± 0.035


## Lexicon counting

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the stopwords
stoplist = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import string

def preprocess(sent, **kwargs):
  sent = sent.lower()
  words = nltk.word_tokenize(sent)

  # Remove Stop words
  words = [w for w in words if w not in stoplist]

  # Remove Punctuation
  words = [w for w in words if w not in string.punctuation]
  
  # Lematization
  if "lemma" in kwargs and kwargs["lemma"]:
    words = [lemmatizer.lemmatize(w) for w in words]
    
  return words

In [ ]:
from sklearn.metrics import f1_score
import sklearn

def predict(words, lexicons):
  cnt = {}
  for label in lexicons:
    cnt[label] = 0

  for w in words:
    for label in lexicons:
      if w in lexicons[label]:
        cnt[label] += lexicons[label][w]

  if sum(cnt.values())==0:
    return None
  
  return max(cnt.items(), key=lambda k: k[1])[0]


def run_eval(data, lexicons, **kwargs):
  texts = data["text"].values
  sents = [preprocess(t, **kwargs) for t in texts]
  labels = data[target_column].values

  y_pred = []
  y_test = []
  non = 0
  for words, label in zip(sents, labels):
    pred = predict(words, lexicons)
    if pred is None:
      non += 1
      continue

    y_pred.append(pred)
    y_test.append(label)

  print(f"F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
  p, r, f1, sup = sklearn.metrics.precision_recall_fscore_support(y_test, y_pred, average="macro")
  print(f"P {p:.3f}")
  print(f"R {r:.3f}")

  print(f"Skip: {non/len(sents):.3f}")
  return

In [ ]:
print("Evaluate PMI Lexicons")
run_eval(test, lexicon_pmi, lemma=True)

Evaluate PMI Lexicons
F1: 0.242
P 0.242
R 0.246
Skip: 0.453


In [ ]:
print("Evaluate Shapley Lexicons")
run_eval(test, lexicon_shapley, lemma=False)

Evaluate Shapley Lexicons
F1: 0.265
P 0.284
R 0.310
Skip: 0.110


In [ ]:
# print("Evaluate Augmented Shapley Lexicons")
# run_eval(test, lexicon_shapley_augmented_v1, lemma=False)

In [ ]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v2_shap, lemma=False)

Evaluate Augmented Shapley Lexicons
F1: 0.265
P 0.283
R 0.310
Skip: 0.107


In [ ]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v3_gptj, lemma=False)

Evaluate Augmented Shapley Lexicons
F1: 0.257
P 0.279
R 0.297
Skip: 0.082


In [ ]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v4_tweetbert, lemma=False)



Evaluate Augmented Shapley Lexicons
F1: 0.265
P 0.284
R 0.310
Skip: 0.110


## Logistic Regression with selected lexicons

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

class Preprocessor:
  def __init__(self, vocabs):
    self.vocabs = vocabs
  
  def run(self, sent):
    words = preprocess(sent, lemma=True)

    # Remove words that are not in vocabs
    words = [w for w in words if w in self.vocabs]
    return words

def dummy_word_tokenize(sent):
  return sent

def train_reg(data, lexicon, **kwargs):
  vocabs = set()
  for label in lexicon:
    vocabs.update(lexicon[label].keys())

  preprocessor = Preprocessor(vocabs)
  vectorizer = CountVectorizer(
      preprocessor=preprocessor.run,
      tokenizer=dummy_word_tokenize
  )

  X_train = vectorizer.fit_transform(data["text"].values)

  y_train = [label_map[l] for l in data[target_column].values]
  
  print("#Vocab", len(vectorizer.get_feature_names_out()))

  model = LogisticRegression(random_state=42, penalty="l2")
  model.fit(X_train, y_train)

  return model, vectorizer


In [ ]:
def run_eval_feat(data, model, vectorizer, **kwargs):
  y_test = [label_map[l] for l in data[target_column].values]
  X_test = vectorizer.transform(data["text"].values)

  y_predict = model.predict(X_test)
  print("F1: %.3f" % sklearn.metrics.f1_score(y_test, y_predict, average='macro'))
  print("Acc: %.3f" % sklearn.metrics.accuracy_score(y_test, y_predict))

  p, r, f1, sup = sklearn.metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")
  print("P: %.3f" % p)
  print("R: %.3f" % r)
  return 

In [ ]:
print("Evaluate PMI Lexicons")
model, vectorizer = train_reg(train, lexicon_pmi)
run_eval_feat(test, model, vectorizer)

Evaluate PMI Lexicons
#Vocab 4556
F1: 0.232
Acc: 0.471
P: 0.382
R: 0.265


In [ ]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley)
run_eval_feat(test, model, vectorizer)


Evaluate Shapley Lexicons
#Vocab 1037
F1: 0.324
Acc: 0.521
P: 0.514
R: 0.325


In [ ]:
# print("Evaluate Shapley Lexicons")
# model, vectorizer = train_reg(train, lexicon_shapley_augmented_v1)
# run_eval_feat(test, model, vectorizer)

In [ ]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v2_shap)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 1260
F1: 0.330
Acc: 0.524
P: 0.508
R: 0.329


In [ ]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v3_gptj)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 1251
F1: 0.319
Acc: 0.510
P: 0.451
R: 0.320


In [ ]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v4_tweetbert)
run_eval_feat(test, model, vectorizer)


Evaluate Shapley Lexicons
#Vocab 1037
F1: 0.324
Acc: 0.521
P: 0.514
R: 0.325


In [ ]:
# lexicon_shapley_augmented_v4_tweetbert.keys()

In [ ]:
A = set(lexicon_shapley_augmented_v4_tweetbert["1. threats, plans to harm and incitement"].keys())

In [ ]:
B = set(lexicon_shapley["1. threats, plans to harm and incitement"].keys())

In [ ]:
len(A.intersection(B)), len(A), len(B)

(174, 445, 174)